In [1]:
import torch
import torch.nn as nn
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch.optim as optim
from ANN import *
from sklearn.model_selection import train_test_split


In [2]:
df_cleaned=pd.read_csv('Processed_DATA_PITSTRAT.csv')
print(df_cleaned.head())

   raceId  driverId  stop  lap  resultId  constructorId  grid  position  \
0     841        17     1   11     20783              9     3       5.0   
1     841         4     1   12     20782              6     5       4.0   
2     841        20     1   14     20779              9     1       1.0   
3     841         1     1   16     20780              1     2       2.0   
4     841       808     1   16     20781              4     6       3.0   

   points  laps  rank  statusId  wins  year  circuitId  
0    10.0    58     3         1     1  2011          1  
1    12.0    58     2         1     0  2011          1  
2    25.0    58     4         1     1  2011          1  
3    18.0    58     8         1     0  2011          1  
4    15.0    58     7         1     0  2011          1  


In [3]:

def preprocess(df_cleaned):

    input_data = df_cleaned[['circuitId', 'raceId', 'points', 'lap','position','laps']].values
    target_data = df_cleaned['stop'].values

    scaler = StandardScaler()
    input_data[:, 2:] = scaler.fit_transform(input_data[:, 2:])

    circuit_encoded = F.one_hot(torch.tensor(input_data[:, 0],dtype=torch.long), num_classes=81)
    race_encoded = F.one_hot(torch.tensor(input_data[:, 1],dtype=torch.long), num_classes=1133)
    points = torch.tensor(input_data[:, 2], dtype=torch.float32).unsqueeze(1)
    lap = torch.tensor(input_data[:, 3], dtype=torch.float32).unsqueeze(1)
    position = torch.tensor(input_data[:, 4], dtype=torch.float32).unsqueeze(1)
    laps = torch.tensor(input_data[:, 5], dtype=torch.float32).unsqueeze(1)


    input_data_encoded = torch.cat([circuit_encoded, race_encoded, points, lap,position,laps], dim=1)
    target_encoded = F.one_hot(torch.tensor(target_data,dtype=torch.long), num_classes=8)
    return input_data_encoded,target_encoded






In [4]:
data = {
    'raceId': [1001],
    'driverId': [21],  # Not used in preprocessing
    'stop': [1],  # Target variable
    'lap': [15],
    'resultId': [10001],  # Not used
    'constructorId': [2],  # Not used
    'grid': [5],  # Not used
    'position': [1],
    'points': [25.0],
    'laps': [70],
    'rank': [1],  # Not used
    'statusId': [1],  # Not used
    'wins': [1],  # Not used
    'year': [20],  # Not used
    'circuitId': [1]
}
df_sample = pd.DataFrame(data)
input_data_encoded,_=preprocess(df_sample)

device='cpu'
race_id_max=1133
circuit_id_max=81
circuit_onehot = input_data_encoded[:, :circuit_id_max]
race_onehot = input_data_encoded[:, circuit_id_max:circuit_id_max + race_id_max]
points_wins = input_data_encoded[:, circuit_id_max + race_id_max:]
race_onehot=race_onehot
circuit_onehot=circuit_onehot
points_wins=points_wins

outputs = Pitstrat_predictor_model(circuit_onehot, race_onehot, points_wins)
softmax=nn.Softmax(dim=1)
probabilities=softmax(outputs)
max_prob_index = torch.argmax(probabilities, dim=1).item()+1
print(max_prob_index)




NameError: name 'Pitstrat_predictor_model' is not defined

In [ ]:
Pitstrat_predictor_model= MLPModel(circuit_id_max=81, race_id_max=1133,dropout_prob=0.2).to(device)
state_dict=(torch.load('../Pitstrat/best_model.pth',map_location=torch.device('cpu')))
Pitstrat_predictor_model.load_state_dict(state_dict)
Pitstrat_predictor_model.eval( )



/tmp/ipykernel_418456/2795656738.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=(torch.load('../Pitstrat/best_model.pth',map_location=torch.device('cpu')))


MLPModel(
  (fc1): Linear(in_features=1218, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=16, bias=True)
  (fc6): Linear(in_features=16, out_features=8, bias=True)
  (elu): ELU(alpha=1.0)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn6): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
import joblib
lr = joblib.load('../lr_model.pkl')

In [ ]:
def align_features(X_train, new_input):
        """
        Align new input features with the training feature space.
        Missing columns will be filled with 0.
        """
        # Identify missing columns between the training and new input
        missing_cols = list(set(X_train.columns) - set(new_input.columns))
        
        # Add missing columns (initialized with 0)
        missing_df = pd.DataFrame(0, index=new_input.index, columns=missing_cols)

        # Combine the original input with the missing columns
        aligned_input = pd.concat([new_input, missing_df], axis=1)

        # Ensure the order of columns matches the training data
        aligned_input = aligned_input[X_train.columns]

        return aligned_input 

In [ ]:
df_lr=pd.read_csv('../data_v1/df_training.csv')

In [ ]:
X = df_lr.drop(['positionOrder'], axis=1)
Y = df_lr['positionOrder']

In [ ]:
X_train_lr, X_test_lr, Y_train_lr, Y_test_lr = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
new_race = {
    'grid': [4, 2,1,6,5,7,11,8,13,10,12,18,3,16,15,21,22,17,20,14,9,19],
    'laps': [57] * 22,  # Same laps for all drivers
    'circuitId_3': [1] * 22 , # Circuit ID 3 for all drivers
    'numStops':[10]*22
}
# One-hot encode driver and constructor IDs
drivers = [8,13,9,2,5,15,17,3,10,4,22,21,1,6,7,19,11,14,16,12,18,20]
constructors = [6,6,2,2,1,7,9,3,7,4,11,10,1,3,5,8,8,9,10,4,11,5]

# Add one-hot encoded driver and constructor columns
for driver in drivers:
    new_race[f'driverId_{driver}'] = [1 if i == drivers.index(driver) else 0 for i in range(22)]

for constructor in constructors:
    new_race[f'constructorId_{constructor}'] = [1 if i == constructors.index(constructor) else 0 for i in range(22)]


new_race_df = pd.DataFrame(new_race)
new_race_df['numStops']=20


aligned_input = align_features(X_train_lr, new_race_df)

predictions = lr.predict(aligned_input)
import numpy as np
rounded_predictions = np.round(predictions, 2)
print(rounded_predictions)
winner_index = predictions.argmax()  
winning_driver = drivers[winner_index]

print(f"The predicted winner is Driver ID: {winning_driver}")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
The predicted winner is Driver ID: 8


In [ ]:
# circuit_onehot = input_data_encoded[:, :circuit_id_max]
# race_onehot = input_data_encoded[:, circuit_id_max:circuit_id_max + race_id_max]
# points_wins = input_data_encoded[:, circuit_id_max + race_id_max:]
# race_onehot=race_onehot.to(device='cuda')
# circuit_onehot=circuit_onehot.to(device='cuda')
# points_wins=points_wins.to(device='cuda')

# outputs = Pitstrat_predictor_model(circuit_onehot, race_onehot, points_wins)
# softmax=nn.Softmax(dim=1)
# probabilities=softmax(outputs)
# max_prob_index = torch.argmax(probabilities, dim=1).item()+1
# print(max_prob_index)



class Winner_predictor_withgivenStrats():
    
    def __init__(self,pitstrat_model,winner_predictor,data_strat,data_winner):
        self.pitstrat_model=pitstrat_model
        self.winner_predicto=winner_predictor
        self.circuit_id_max = 81
        self.race_id_max = 1133
        self.data1=data_strat
        self.data2=data_winner
        
    def preprocess(self):
        df_cleaned=self.data1
        input_data = df_cleaned[['circuitId', 'raceId', 'points', 'lap','position','laps']].values

        scaler = StandardScaler()
        input_data[:, 2:] = scaler.fit_transform(input_data[:, 2:])

        circuit_encoded = F.one_hot(torch.tensor(input_data[:, 0],dtype=torch.long), num_classes=81)
        race_encoded = F.one_hot(torch.tensor(input_data[:, 1],dtype=torch.long), num_classes=1133)
        points = torch.tensor(input_data[:, 2], dtype=torch.float32).unsqueeze(1)
        lap = torch.tensor(input_data[:, 3], dtype=torch.float32).unsqueeze(1)
        position = torch.tensor(input_data[:, 4], dtype=torch.float32).unsqueeze(1)
        laps = torch.tensor(input_data[:, 5], dtype=torch.float32).unsqueeze(1)


        input_data_encoded = torch.cat([circuit_encoded, race_encoded, points, lap,position,laps], dim=1)
        return input_data_encoded
    
    
    def predict_pitstop(self):
        df_cleaned=self.data1
        print(self.circuit_id_max,self.race_id_max)
        input_data_encoded=self.preprocess()
        
        circuit_onehot = input_data_encoded[:, :self.circuit_id_max]
        race_onehot = input_data_encoded[:, self.circuit_id_max:self.circuit_id_max + self.race_id_max]
        points_wins = input_data_encoded[:, self.circuit_id_max + self.race_id_max:]
        race_onehot=race_onehot
        circuit_onehot=circuit_onehot
        points_wins=points_wins

        outputs = Pitstrat_predictor_model(circuit_onehot, race_onehot, points_wins)
        softmax=nn.Softmax(dim=1)
        probabilities=softmax(outputs)
        return torch.argmax(probabilities, dim=1).item()+1
    
    def align_features(X_train, new_input):
        """
        Align new input features with the training feature space.
        Missing columns will be filled with 0.
        """
        missing_cols = list(set(X_train.columns) - set(new_input.columns))
        
        missing_df = pd.DataFrame(0, index=new_input.index, columns=missing_cols)

        aligned_input = pd.concat([new_input, missing_df], axis=1)

        aligned_input = aligned_input[X_train.columns]

        return aligned_input    
    
    def predict_winner(self,predicted_optimal_stop):
        df_lr=pd.read_csv('../data_v1/df_training.csv')
        X = df_lr.drop(['positionOrder'], axis=1)
        Y = df_lr['positionOrder']
        
        X_train_lr, X_test_lr, Y_train_lr, Y_test_lr = train_test_split(X, Y, test_size=0.2, random_state=42)
        for driver in drivers:
            self.data2[f'driverId_{driver}'] = [1 if i == drivers.index(driver) else 0 for i in range(22)]

        for constructor in constructors:
            self.data2[f'constructorId_{constructor}'] = [1 if i == constructors.index(constructor) else 0 for i in range(22)]

        aligned_input = align_features(X_train_lr, new_race_df)

        self.data2_df = pd.DataFrame(self.data2)
        self.data2_df['numStops']=predicted_optimal_stop
        predictions = self.winner_predicto.predict(aligned_input)
        winner_index = predictions.argmax()  
        winning_driver = drivers[winner_index]

        print(f"The predicted winner is Driver ID: {winning_driver}")
    
    def forward_pass(self):
        optimal_pitstop=self.predict_pitstop()
        print(f"The optimal number of pitstop is: {winning_driver}")
        self.predict_winner(optimal_pitstop)
        
    
    
    

In [ ]:
new_race = {
    'grid': [4, 2,1,6,5,7,11,8,13,10,12,18,3,16,15,21,22,17,20,14,9,19],
    'laps': [57] * 22,  # Same laps for all drivers
    'circuitId_3': [1] * 22 , # Circuit ID 3 for all drivers
    'numStops':[10]*22
}
# One-hot encode driver and constructor IDs
drivers = [8,13,9,2,5,15,17,3,10,4,22,21,1,6,7,19,11,14,16,12,18,20]
constructors = [6,6,2,2,1,7,9,3,7,4,11,10,1,3,5,8,8,9,10,4,11,5]

# Add one-hot encoded driver and constructor columns
for driver in drivers:
    new_race[f'driverId_{driver}'] = [1 if i == drivers.index(driver) else 0 for i in range(22)]

for constructor in constructors:
    new_race[f'constructorId_{constructor}'] = [1 if i == constructors.index(constructor) else 0 for i in range(22)]


new_race_df = pd.DataFrame(new_race)


data = {
    'raceId': [1001],
    'driverId': [21],  # Not used in preprocessing
    'stop': [1],  # Target variable
    'lap': [15],
    'resultId': [10001],  # Not used
    'constructorId': [2],  # Not used
    'grid': [5],  # Not used
    'position': [3],
    'points': [25.0],
    'laps': [30],
    'rank': [1],  # Not used
    'statusId': [1],  # Not used
    'wins': [1],  # Not used
    'year': [20],  # Not used
    'circuitId': [1]
}
df_sample = pd.DataFrame(data)
input_data_encoded,_=preprocess(df_sample)

In [ ]:
Final_Pipe=Winner_predictor_withgivenStrats(Pitstrat_predictor_model,lr,df_sample,new_race_df)
Final_Pipe.forward_pass()

81 1133
The optimal number of pitstop is: 8
The predicted winner is Driver ID: 8
